### Data Science Captstone Project

Used for neighborhood segementation and clustering


In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Week3 - Task 1

Importing Beautiful Soup 

Downloading the respective table and transforming it into a Pandas DataFrme

In [3]:
import bs4 as bs
import urllib.request


source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source)

table = soup.find_all('table',{"class": "wikitable"})
df = pd.read_html(str(table))[0]

Deleting all Rows that do not have an assigned Borough

In [4]:
df = df[df["Borough"]!= "Not assigned"]

In the current version the steps of joining duplicate postal codes and adding missing neighborhoods do not need to be done because it is already cleaned within wikipedia itself

In [5]:
if len(df["Postal code"].unique())-len(df)==0:
    print("No Duplicate postal codes in current table version")

No Duplicate postal codes in current table version


In [6]:
if len(df[df["Neighborhood"]== np.nan])==0: 
    print("All Boroughs have an assigned Neighborhood")

All Boroughs have an assigned Neighborhood


In [7]:
df.shape

(103, 3)

# Week3 - Task 2

### Download geodata and merge it based on Postal Code

In [8]:
geodata=pd.read_csv('https://cocl.us/Geospatial_data').rename(columns={"Postal Code": "Postal code"})

In [9]:
df=df.merge(geodata, on="Postal code")

In [10]:
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509


# Week3 - Task 3

#### Connect to Foursquare API and download relevant libraries

In [11]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests
from pandas.io.json import json_normalize

In [12]:
CLIENT_ID = 'FQ2IDDRDQHPLRZPINBZP2U2NMNIRUV0RZ01Z4EPZKQGM13DM' # your Foursquare ID
CLIENT_SECRET = 'ZCMNXOPSI15W1OQ0WAKVOEREL0MYYJBY2PRBCVRUMHAQOYW3' # your Foursquare Secret

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FQ2IDDRDQHPLRZPINBZP2U2NMNIRUV0RZ01Z4EPZKQGM13DM
CLIENT_SECRET:ZCMNXOPSI15W1OQ0WAKVOEREL0MYYJBY2PRBCVRUMHAQOYW3


In [31]:
df["Number_Bars"]= 0

#### Run query - and get number of Bars within 1km cirumferenc of Borough center, add number of Bars to DataFrame. 

Include number of Bars as future cluster in the folium label

In [27]:
search_query = '4d4b7105d754a06376d81259'
radius = 1000
VERSION = '20180604'


for row in df.itertuples():
    index = row[0]
    latitude = row[4]
    longitude = row[5]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df["Number_Bars"][index]=len(venues)
    
    
df

/Users/tobiasuthe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Postal code,Borough,Neighborhood,Latitude,Longitude,Number of Bars,Number_Bars
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,5,5
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,29,29
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,9,9
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,30,30
...,...,...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944,13,13
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,30,30
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558,28,28
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509,0,0


In [32]:
from geopy.geocoders import Nominatim 
import folium
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood, nightlife in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'], df["Number_Bars"]):
    label = '{}, {}, {}'.format(neighborhood, borough, nightlife)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  


map_toronto

GeocoderTimedOut: Service timed out